# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [31]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

import warnings
warnings.filterwarnings('ignore')

# Import API key
from api_keys import geoapify_key

In [32]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head(20)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port mathurin,-19.6833,63.4167,21.38,74,11,11.67,MU,2024-09-16 16:44:36
1,1,emerald,-23.5333,148.1667,15.19,61,18,2.88,AU,2024-09-16 16:51:24
2,2,wailua homesteads,22.0669,-159.3780,25.44,88,75,5.66,US,2024-09-16 16:51:25
3,3,puerto ayora,-0.7393,-90.3518,21.97,91,91,2.68,EC,2024-09-16 16:51:26
4,4,port-aux-francais,-49.3500,70.2167,4.31,94,100,15.52,TF,2024-09-16 16:44:14
5,5,bredasdorp,-34.5322,20.0403,10.34,77,0,2.36,ZA,2024-09-16 16:51:28
6,6,umluj,25.0213,37.2685,30.08,71,0,3.62,SA,2024-09-16 16:51:29
7,7,uturoa,-16.7333,-151.4333,25.36,80,76,4.72,PF,2024-09-16 16:51:31
8,8,lebu,-37.6167,-73.6500,12.91,61,100,7.18,CL,2024-09-16 16:51:32
9,9,ta`u,-14.2336,-169.5144,26.44,76,8,3.12,AS,2024-09-16 16:51:33


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [33]:
%%capture --no-display

# Configure the map plot

humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    alpha = 0.7

)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [34]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df.count()

city_data_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= 25) & 
    (city_data_df["Max Temp"] >= 18) & 
    (city_data_df["Humidity"] <= 40) &
    (city_data_df["Cloudiness"] <= 25) &
    (city_data_df["Wind Speed"] <= 5)
]

# Drop any rows with null values
city_data_df = city_data_df.dropna()


# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
69,69,mazyr,52.0495,29.2456,20.79,35,3,4.01,BY,2024-09-16 16:52:42
262,262,turkmenbasy,40.0222,52.9552,21.43,18,0,3.09,TM,2024-09-16 16:56:33
280,280,hami,42.8000,93.4500,22.23,19,0,2.76,CN,2024-09-16 16:56:56
304,304,delijan,33.9905,50.6838,24.44,31,4,1.81,IR,2024-09-16 16:57:25
371,371,castlegar,49.2998,-117.6689,18.50,38,1,1.27,CA,2024-09-16 16:55:01


### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = city_data_df.copy(deep=True)


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
69,69,mazyr,52.0495,29.2456,20.79,35,3,4.01,BY,2024-09-16 16:52:42,
262,262,turkmenbasy,40.0222,52.9552,21.43,18,0,3.09,TM,2024-09-16 16:56:33,
280,280,hami,42.8000,93.4500,22.23,19,0,2.76,CN,2024-09-16 16:56:56,
304,304,delijan,33.9905,50.6838,24.44,31,4,1.81,IR,2024-09-16 16:57:25,
371,371,castlegar,49.2998,-117.6689,18.50,38,1,1.27,CA,2024-09-16 16:55:01,
375,375,kasulu,-4.5767,30.1025,23.76,29,13,2.20,TZ,2024-09-16 16:58:53,
420,420,hotan,37.0997,79.9269,23.31,26,10,1.69,CN,2024-09-16 16:59:48,
535,535,prieska,-29.6641,22.7474,20.86,19,0,4.04,ZA,2024-09-16 17:02:01,
546,546,kashgar,39.4547,75.9797,23.56,31,20,3.00,CN,2024-09-16 17:02:14,
559,559,klichaw,53.4928,29.3352,19.11,35,2,3.66,BY,2024-09-16 17:02:30,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [36]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
categories = "accommodation.hotel"
params = {
    "limit":limit,
    "apiKey":geoapify_key,
    "categories": categories
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index,"Lng"]
    latitude = hotel_df.loc[index,"Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
        
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mazyr - nearest hotel: Прыпяць
turkmenbasy - nearest hotel: Türkmenbaşy Oteli
hami - nearest hotel: Хами
delijan - nearest hotel: هتل سبز کارون
castlegar - nearest hotel: Sandman Hotel Castlegar
kasulu - nearest hotel: Alfa Lodge
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
prieska - nearest hotel: No hotel found
kashgar - nearest hotel: 深航国际酒店
klichaw - nearest hotel: Ольса
laramie - nearest hotel: Laramie Valley Inn


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
69,69,mazyr,52.0495,29.2456,20.79,35,3,4.01,BY,2024-09-16 16:52:42,Прыпяць
262,262,turkmenbasy,40.0222,52.9552,21.43,18,0,3.09,TM,2024-09-16 16:56:33,Türkmenbaşy Oteli
280,280,hami,42.8000,93.4500,22.23,19,0,2.76,CN,2024-09-16 16:56:56,Хами
304,304,delijan,33.9905,50.6838,24.44,31,4,1.81,IR,2024-09-16 16:57:25,هتل سبز کارون
371,371,castlegar,49.2998,-117.6689,18.50,38,1,1.27,CA,2024-09-16 16:55:01,Sandman Hotel Castlegar
375,375,kasulu,-4.5767,30.1025,23.76,29,13,2.20,TZ,2024-09-16 16:58:53,Alfa Lodge
420,420,hotan,37.0997,79.9269,23.31,26,10,1.69,CN,2024-09-16 16:59:48,Hua Yi International Hotel (accepts foreigners)
535,535,prieska,-29.6641,22.7474,20.86,19,0,4.04,ZA,2024-09-16 17:02:01,No hotel found
546,546,kashgar,39.4547,75.9797,23.56,31,20,3.00,CN,2024-09-16 17:02:14,深航国际酒店
559,559,klichaw,53.4928,29.3352,19.11,35,2,3.66,BY,2024-09-16 17:02:30,Ольса


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [37]:
%%capture --no-display

# Configure the map plot

hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = 100,
    color = "City",
    alpha = 1,
    hover_cols = ["Hotel Name","Country"]

)

# Display the map
hotel_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)